In [1]:
import pandas as pd
import os
from main import get_miner
from log_parsing import from_id_to_template
import ast

In [2]:
df = pd.read_csv("output/features_labeled_v3.csv")

In [3]:
df.head()

,normal_time,abnormal_time,result_list,inject_time,inject_pod,inject_type,root_cause
0,2022-08-22 03:51,2022-08-22 03:54,"[{'events': '76_4', 'score': 1.0, 'deepth': 1,...",2022-08-22 03:53:54,frontend-579b9bff58-t2dbm,cpu_contention,['CpuUsageRate(%)']
1,2022-08-22 03:51,2022-08-22 03:55,"[{'events': '76_4', 'score': 1.0, 'deepth': 1,...",2022-08-22 03:53:54,frontend-579b9bff58-t2dbm,cpu_contention,['CpuUsageRate(%)']
2,2022-08-22 03:51,2022-08-22 04:03,"[{'events': '120_198', 'score': 1.0, 'deepth':...",2022-08-22 04:02:07,frontend-579b9bff58-t2dbm,return,"['Serving product page started', 'GetProduct s..."
3,2022-08-22 03:51,2022-08-22 04:04,"[{'events': '120_198', 'score': 1.0, 'deepth':...",2022-08-22 04:02:07,frontend-579b9bff58-t2dbm,return,"['Serving product page started', 'GetProduct s..."
4,2022-08-22 03:51,2022-08-22 04:11,"[{'events': '76_4', 'score': 1.0, 'deepth': 1,...",2022-08-22 04:10:20,frontend-579b9bff58-t2dbm,cpu_consumed,['CpuUsageRate(%)']


In [4]:
df['result_list'] = df['result_list'].apply(lambda x: ast.literal_eval(x))
df['root_cause'] = df['root_cause'].apply(lambda x: ast.literal_eval(x))

In [20]:
mask = df.result_list.apply(lambda x: any(['resource_alert' in y and "Net" in y['resource_alert'] for y in x]))

In [21]:
pd.set_option('display.max_colwidth', 1000)
df[mask].head()

,normal_time,abnormal_time,result_list,inject_time,inject_pod,inject_type,root_cause


In [5]:
df['inject_type'].unique()

array(['cpu_contention', 'return', 'cpu_consumed', 'exception',
       'network_delay'], dtype=object)

In [6]:
df['root_cause'].unique()

array(["['CpuUsageRate(%)']",
       "['Serving product page started', 'GetProduct start']",
       "['Placing order started', 'Order placed complete']",
       "['NetworkP90(ms)']",
       "['Start charge card', 'Charge successfully']",
       "['Received ad request', 'No context provided']",
       "['Query product with name', 'Query product successfully']"],
      dtype=object)

In [26]:
log_template_miner = get_miner()

In [53]:
def get_nezha_rank(result_list, root_cause, inject_pod):
    rank = 1
    rc_found_flag = False
            
    if len(root_cause) == 1:
        for i in range(len(result_list)):
            if "resource_alert" in result_list[i].keys():
                if str(root_cause[0]) in str(result_list[i]["resource_alert"]) and inject_pod in str(result_list[i]["pod"]):
                    rc_found_flag = True
                    break
            else:
                if "events_actual" in result_list[i].keys() and str(root_cause[0]) in from_id_to_template(int(result_list[i]["events_actual"].split("_")[1]),log_template_miner) and inject_pod in str(result_list[i]["pod"]):
                    rc_found_flag = True
                    break
                if i > 0:
                    if result_list[i-1]["score"] == result_list[i]["score"] and result_list[i-1]["deepth"] == result_list[i]["deepth"]:
                        continue
                    else:
                        rank += 1
                elif i == 0:
                    rank += 1
    elif len(root_cause) == 2:
        for i in range(len(result_list)):
            if root_cause[0] in from_id_to_template(int(result_list[i]["events"].split(
                    "_")[0]),log_template_miner) and root_cause[1] in from_id_to_template(int(result_list[i]["events"].split("_")[1]),log_template_miner) and inject_pod in str(result_list[i]["pod"]):
                rc_found_flag = True
                break
            else:
                if i > 0:
                    # logger.info("%s, %s",
                    #             result_list[i-1]["score"], result_list[i]["score"])
                    if result_list[i-1]["score"] == result_list[i]["score"] and result_list[i-1]["deepth"] == result_list[i]["deepth"]:
                        continue
                    else:
                        rank += 1
                elif i == 0:
                    rank += 1
    
    if not rc_found_flag:
        return -1
    return rank

In [54]:
from_id_to_template(199,log_template_miner)

'NetworkP90(ms)'

In [55]:
df["nezha_rank"] = df.apply(lambda row: get_nezha_rank(row['result_list'], row['root_cause'], row['inject_pod']), axis=1)

In [56]:
df.nezha_rank.value_counts()

 1    98
-1     9
 2     5
Name: nezha_rank, dtype: int64

In [59]:
98/112

0.875

In [57]:
mask = df.nezha_rank == -1
pd.set_option('display.max_colwidth', None)
df[mask][['result_list', 'root_cause']]

,result_list,root_cause
18,[],"[Start charge card, Charge successfully]"
72,[],[CpuUsageRate(%)]
82,[],[NetworkP90(ms)]
83,"[{'events': '24_100', 'score': 0.7, 'deepth': 3, 'pod': 'currencyservice-cf787dd48-vpjrd'}]",[NetworkP90(ms)]
84,"[{'events': '198_120', 'score': 0.81, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resource_alert': 'MemoryUsageRate(%)'}, {'events': '28_119', 'score': 0.81, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resource_alert': 'MemoryUsageRate(%)'}, {'events': '18_17', 'score': 0.81, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_192'}, {'events': '18_80', 'score': 0.81, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_192'}, {'events': '18_82', 'score': 0.81, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_192'}, {'events': '18_84', 'score': 0.81, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_192'}, {'events': '18_86', 'score': 0.81, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_192'}, {'events': '16_15', 'score': 0.81, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm'}, {'events': '120_198', 'score': 0.7904040404040404, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resource_alert': 'MemoryUsageRate(%)'}, {'events': '27_26', 'score': 0.7904040404040404, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resource_alert': 'MemoryUsageRate(%)'}]","[Query product with name, Query product successfully]"
88,"[{'events': '38_84', 'score': 0.9831081081081081, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '38_195'}, {'events': '38_37', 'score': 0.9805825242718447, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '38_195'}, {'events': '38_124', 'score': 0.9805825242718447, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '38_195'}, {'events': '198_120', 'score': 0.9759036144578314, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resource_alert': 'MemoryUsageRate(%)'}, {'events': '28_119', 'score': 0.9759036144578314, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resource_alert': 'MemoryUsageRate(%)'}, {'events': '18_17', 'score': 0.9759036144578314, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_192'}, {'events': '18_80', 'score': 0.9759036144578314, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_192'}, {'events': '18_82', 'score': 0.9759036144578314, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_192'}, {'events': '18_84', 'score': 0.9759036144578314, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_192'}, {'events': '18_86', 'score': 0.9759036144578314, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_192'}]","[Query product with name, Query product successfully]"
102,[],[NetworkP90(ms)]
103,"[{'events': '24_100', 'score': 0.7, 'deepth': 3, 'pod': 'currencyservice-cf787dd48-vpjrd'}, {'events': '23_100', 'score': 0.6744186046511628, 'deepth': 3, 'pod': 'currencyservice-cf787dd48-vpjrd'}]",[NetworkP90(ms)]
106,[],"[Received ad request, No context provided]"


In [11]:
def format_result_list(result_list):
    """
    Format result list for LLM comprehension
    """
    formatted_entries = [
        f"Ranked ambigous patterns (by comparing normal event traces with current faulty event traces)"
    ]
    for i in range(len(result_list)):
        entry = result_list[i]
        entry_details = [
            f"Pattern {1}:",
            f"- Expected Event Trace:",
            f"    From: {from_id_to_template(int(entry['events'].split('_')[0]), log_template_miner)}",
            f"    To: {from_id_to_template(int(entry['events'].split('_')[1]), log_template_miner)}",
        ]
        
        if 'events_actual' in entry:
            entry_details.extend([
            f"- Actual Event Trace:",
            f"    From: {from_id_to_template(int(entry['events_actual'].split('_')[0]), log_template_miner)}",
            f"    To: {from_id_to_template(int(entry['events_actual'].split('_')[1]), log_template_miner)}",
        ])
        
        entry_details.extend([
            
            f"- Abnormal Score: {entry.get('score', 'N/A')}"
            f"- Trace Depth: {entry.get('deepth', 'N/A')}"
            f"- Affected Pod: {entry.get('pod', 'N/A')}"
        ])
        
        if 'resource_alert' in entry:
            entry_details.append(f"- Resource Alert: {entry['resource_alert']}")
        
        formatted_entries.append("\n".join(entry_details))
    
    return "\n".join(formatted_entries)

In [12]:
df['result_list_string'] = df['result_list'].apply(format_result_list)

In [14]:
pd.set_option('display.max_colwidth', None)
df[['result_list_string']].head()

result_list_string
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    Ranked ambigous patterns (by comparing normal event traces with current faulty event traces)\nPattern 1:\n- Expected Event Trace:\n    From: frontend hipstershop.Frontend/Recv. start\n    To: TraceID: <:TRACEID:> SpanID: <:SPANID:> Request started\n- Actual Event Trace:\n    From: frontend hipstershop.Frontend/Recv. start\n    To: CpuUsageRate(%)\n- Abnormal Score: 1.0- Tra

In [15]:
df.columns

Index(['normal_time', 'abnormal_time', 'result_list', 'inject_time',
       'inject_pod', 'inject_type', 'root_cause', 'result_list_string'],
      dtype='object')

In [16]:
df.drop(columns="result_list").to_csv("output/features_labeled_4llm_v2.csv", index=False)